In [2]:
class KnapsackNode:
    def __init__(self, weight, value, parent=None, action=0):
        self.weight = weight
        self.value = value
        self.parent = parent
        self.action = action
        self.g_score = 0
        self.h_score = 0
        self.f_score = self.g_score + self.h_score

    def __eq__(self, other):
        return self.weight == other.weight and self.value == other.value

    def __hash__(self):
        return hash((self.weight, self.value))

    def calculate_heuristic(self, capacity, items):
        remaining_capacity = capacity - self.weight
        remaining_items = items[self.action + 1:]
        estimated_value = self.value
        estimated_weight = self.weight

        for weight, value in remaining_items:
            if weight <= remaining_capacity:
                estimated_value += value
                estimated_weight += weight
                remaining_capacity -= weight
            else:
                estimated_value += (remaining_capacity / weight) * value
                break

        return estimated_value


class Knapsack:
    def __init__(self, capacity, items):
        self.capacity = capacity
        self.items = items

    def generate_next_states(self, node):
        next_states = []
        for i, item in enumerate(self.items):
            if i >= node.action:
                weight, value = item
                if node.weight + weight <= self.capacity:
                    next_state = KnapsackNode(node.weight + weight, node.value + value, node, i)
                    next_state.g_score = node.g_score + value
                    next_state.h_score = next_state.calculate_heuristic(self.capacity, self.items)
                    next_state.f_score = next_state.g_score + next_state.h_score
                    next_states.append(next_state)
        return next_states

    def is_goal_state(self, node):
        return node.weight == self.capacity or node.action == len(self.items) - 1


class KnapsackSearch:

    def a_star_search(self, capacity, items):
        knapsack = Knapsack(capacity, items)
        initial_node = KnapsackNode(0, 0)
        open_set = {initial_node}
        closed_set = set()

        while open_set:
            current_node = min(open_set, key=lambda node: node.f_score)
            open_set.remove(current_node)
            closed_set.add(current_node)

            if knapsack.is_goal_state(current_node):
                return self.get_path(current_node)

            for neighbor in knapsack.generate_next_states(current_node):
                if neighbor in closed_set:
                    continue

                if neighbor not in open_set or neighbor.g_score < neighbor.g_score:
                    open_set.add(neighbor)

        return None

    def get_path(self, node):
        path = []
        while node:
            path.append((node.weight, node.value))
            node = node.parent
        return path[::-1]


# Example usage
capacity = 10
items = [(5, 10), (4, 40), (6, 30), (3, 50)]

solution_path = KnapsackSearch().a_star_search(capacity, items)
if solution_path is not None:
    print("Solution Path:")
    for step in solution_path:
        print(step)
else:
    print("A* Search failed to find a solution.")


Solution Path:
(0, 0)
(5, 10)
(10, 20)
